# KneDle: Licitações e Contratos
## DODF Evaluation by RegEx 

<div align="center">Figure 1. WorkFlow

![alternative text](DODF/esquema.png "WorkFlow")

In [3]:
import re
import docx
import pdfplumber
import os

## Regex Class

In [4]:
class TTerm:
    name = ''
    begin = []
    end  = []
    
#Constructor: Update the following properties: 
#             name - String of a term 
#             begin, end - term positions related to input parameter ptext 

    def __init__(self, ptext, pterm):
        self.begin = []
        self.end = []
        self.name = pterm
        pstart = 0
        while (pstart < len(ptext)):
            paux = re.search(pterm, ptext[pstart:])
            if paux is not None:
                pauxbegin, pauxend = paux.span()
                self.begin.append(pauxbegin + pstart)
                self.end.append(pauxend + pstart)
                pstart     = pauxend + pstart
            else: 
                break;

class TExpression:
    name = ''
    terms = []
    
#Constructor: Update the following properties: 
#             name - String of a Regular Expression
#             terms - list of terms found by searching for a RE (name).

    def __init__(self, ptext, pexpression):
        self.name = pexpression
        pauxterms = re.findall(self.name, ptext) #finding all terms related to one RE (pexpression).
        pauxterms = set(pauxterms) # list with unique terms
        self.terms = []
        for pt in pauxterms:
            self.terms.append(TTerm(ptext, pt))
            
class TREClass:
    token = ''
    expressionnamelist = []
    expressions = []
    
#Constructor: Update the following properties: 
#             expressionnamelist - it must hold a list of Regular Expressions.
#             token: it represents all REs in expressionnamelist

    def __init__(self, ptoken, pexpressionnamelist):
        self.token = ptoken
        self.expressionnamelist = pexpressionnamelist
        
#Update the propertie expressions - it must hold the found terms inside the input parameter ptext.
    def applyExpressions(self, ptext): #applying the set of RE on a text.
        self.expressions = []
        for pexp in self.expressionnamelist:
            self.expressions.append(TExpression(ptext, pexp))
    
#Return: A dictionary with regular expressions and its unique terms related to the processed text (after applyExpressions)    
    def getExpressions(self):
        pret = {}
        for pexp in self.expressions:
            pret[pexp.name] = [] #list of terms 
            for pt in pexp.terms:  #Terms of the expression pexp
                if pt.name not in pret[pexp.name]: 
                    pret[pexp.name].append(pt.name)
        return pret

#Return: A dictionary with unique terms and their locations related to the processed text (after applyExpressions)    
    def getTerms(self):
        pret = {}
        for pexp in self.expressions:
            for pt in pexp.terms:
                if pt.name not in pret: 
                    pret[pt.name] = []
                pret[pt.name].append((pt.begin, pt.end))
        return pret

#Return: A score related to the processed text (after applyExpressions): 
#        Proportional to    
    def getScore(self):
        pret = 0
        for pexp in self.expressions:
            for pt in pexp.terms:
                psize = len(pt.name.split()) #term size (in words)
                pfreq = len(pt.begin) # How many times this term occurs
                pret  = pret + psize*pfreq
        return (pret)

## DODF Loading

In [5]:
class TPDFFile:
    fextension = ''
    fname = ''
    filename = ''
    pdf = None
    
    def __init__(self, pfilename):
        auxname, auxextension = os.path.splitext(pfilename)
        if (auxextension == '.pdf'):
            self.fname, self.fextension, self.filename = auxname, auxextension, pfilename
            self.pdf = pdfplumber.open(self.filename)
        
    def getPageText(self, pnumpage):
        PageText = []
        pText = []
        page = self.pdf.pages[pnumpage]
        pText.append(page.extract_text())
        PageText = '\n'.join(pText)
        return PageText

In [6]:
PDFFile = TPDFFile('dodf/DODF_011_16_01_2020_INTEGRA.pdf')
texto = PDFFile.getPageText(80).lower()
print(texto)

nº 11,  quinta-feira, 16 de janeiro de  2020 diário oficial do distrito federal página 81
ceb distribuição s.a.
secretaria de estado de justiça e
edital nº 299 - concurso público nº 1/2012-ceb
c i da da n i a 130ª convocação de candidatos
o diretor-geral da ceb distribuição s/a, no uso de suas atribuições e tendo em vista o constante no
edital n. 01/2012-ceb, edital normativo, publicado no diário oficial do distrito federal em 26.9.2012 e
edital de republicação publicado no diário oficial do distrito federal em 15.10.2013, torna público que
secretaria executiva
convoca os candidatos relacionados abaixo para comparecerem à sede da ceb, localizada no sia -
setor de área pública lote c, bloco d, sala 05 - superintendência de recursos humanos - srh,
extrato do termo aditivo nº 1 ao termo de
brasília/df, no período de 16 a 20 de janeiro de 2020 (dias úteis) no horário de 14h30 as 17h, para
convênio n° 03/2018 convênio n° 03/2018.
manifestação acerca de sua contratação e encaminhamento à ava

* [ ] _What's the best RegExp set (**expvenc**) for "aviso de vencedor"?_ 

In [36]:
#print(texto)
#expvenc = ['aviso de declaração de vencedor', 'após verificada a aceitabilidade', 'aceitabilidade', 'aviso', 'convocatório', 'licitatório', 'vencedor', 'vencedora']
expvenc = [r'\bde\w+', r'\bca\w+']
tokvenc = 'TK_AVISO_VENCEDOR'

In [37]:
#RegExp = TREClass('TK_TESTE', [r'\bd\w+', r'\bC\w+'])
RegExp = TREClass(tokvenc, expvenc)
#print(RegExp.token, len(RegExp.expressionnamelist))
print(RegExp.token, ':', RegExp.expressionnamelist)

TK_AVISO_VENCEDOR : ['\\bde\\w+', '\\bca\\w+']


In [42]:
RegExp.applyExpressions(texto)
#pr = RegExp.getExpressions()
#pr2 = RegExp.getTerms()

In [43]:
print('Token:', RegExp.token)
for pexp in RegExp.expressions:
    print('  \nExpressão:', pexp.name)
    for pt in pexp.terms:
        print('    Termo:', pt.name)
        for pind in range(0,len(pt.begin)):
            print('      Local', pind, ':', pt.begin[pind], '-', pt.end[pind])
             

Token: TK_AVISO_VENCEDOR
  
Expressão: \bde\w+
    Termo: definitiva
      Local 0 : 1449 - 1459
    Termo: delcimar
      Local 0 : 4884 - 4892
    Termo: despesa
      Local 0 : 5533 - 5540
      Local 1 : 5582 - 5589
      Local 2 : 6362 - 6369
      Local 3 : 6422 - 6429
      Local 4 : 7164 - 7171
      Local 5 : 8692 - 8699
      Local 6 : 9599 - 9606
      Local 7 : 10785 - 10792
    Termo: despesas
      Local 0 : 5533 - 5541
      Local 1 : 5582 - 5590
      Local 2 : 6362 - 6370
      Local 3 : 7164 - 7172
      Local 4 : 8692 - 8700
      Local 5 : 9599 - 9607
      Local 6 : 10785 - 10793
    Termo: declaração
      Local 0 : 3080 - 3090
    Termo: denilson
      Local 0 : 4824 - 4832
    Termo: demonstrativo
      Local 0 : 5516 - 5529
      Local 1 : 5565 - 5578
      Local 2 : 6345 - 6358
      Local 3 : 7147 - 7160
    Termo: despesasde
      Local 0 : 8692 - 8702
    Termo: demonstrações
      Local 0 : 10323 - 10336
    Termo: declarada
      Local 0 : 3369 - 3378
   

In [44]:
print('Scores of', RegExp.token, ': Absolute: ', RegExp.getScore(), 'and Relative:', (RegExp.getScore()/len(texto)))

Scores of TK_AVISO_VENCEDOR : Absolute:  54 and Relative: 0.0047062924873627334


## DODF Evaluation by using key terms

---

### Next: 
#### Samples Data achievement:
* [ ] Use "Data Exploration" notebook in order to select key terms for a specific **procedural act** (of "licitação")
* [ ] Access DODFMiner base 
* [ ] Access **pieces of DODF pages** 
* [ ] **Evaluate pieces of DODF**: Apply "RegExp.getScore()" for each pieces of DODF pages.
* [ ] Define a "good" minimum threshold that **identify a piece as a procedural act**.
* [ ] Construct a **database of procedural act samples**.

#### Modelling:
* [ ] **Train an AI model** to identify a procedural act into the DODFMiner.
* [ ] **Integrate the model into the DODFMiner**.